In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/311_Explorer_02.csv', parse_dates=\
                 ['Date Created','Date Closed'],\
                 low_memory = False)
df2 = pd.read_csv('data/weather_categorized.csv', parse_dates=\
                 ['date'],\
                 low_memory = False)
df3 = pd.read_csv('data/council_cleaned.csv', parse_dates=\
                 ['MEETING_DATE'],\
                 low_memory = False)

In [2]:
df3 = df3.drop_duplicates()
df3

,MEETING_TYPE,MEETING_DATE
0,Committee,2013-10-23
1,Council,2013-10-29
2,Committee,2013-10-30
5,Council,2013-10-30
6,Hearing,2013-10-31
7,Hearing,2013-11-04
8,Committee,2013-11-05
9,Council,2013-11-06
10,Council,2013-11-07
11,Committee,2013-11-08


In [3]:
df8 = pd.DataFrame({'MEETING_DATE':df3['MEETING_DATE'].unique()})
df8

,MEETING_DATE
0,2013-10-23
1,2013-10-29
2,2013-10-30
3,2013-10-31
4,2013-11-04
5,2013-11-05
6,2013-11-06
7,2013-11-07
8,2013-11-08
9,2013-11-12


In [4]:
dates = df3['MEETING_DATE'].unique()
for x in dates:
    council = False
    hearing = False
    committee = False
    v = df3.loc[df3['MEETING_DATE'] == x]['MEETING_TYPE'].values
    
    if 'Council' in v:
        council = True
    if 'Hearing' in v:
        hearing = True
    if 'Committee' in v:
        committee = True
        
    if council and hearing and committee:
        df8.loc[df8['MEETING_DATE'] == x, 'MEETING_TYPE'] = 'Council/Hearing/Committee'
    elif hearing and committee:
        df8.loc[df8['MEETING_DATE'] == x, 'MEETING_TYPE'] = 'Hearing/Committee'    
    elif council and hearing:
        df8.loc[df8['MEETING_DATE'] == x, 'MEETING_TYPE'] = 'Council/Hearing'
    elif council and committee:
        df8.loc[df8['MEETING_DATE'] == x, 'MEETING_TYPE'] = 'Council/Committee'
    elif council:
        df8.loc[df8['MEETING_DATE'] == x, 'MEETING_TYPE'] = 'Council'
    elif hearing:
        df8.loc[df8['MEETING_DATE'] == x, 'MEETING_TYPE'] = 'Hearing'
    elif committee:
        df8.loc[df8['MEETING_DATE'] == x,'MEETING_TYPE'] = 'Committee'        

In [5]:
df8['MEETING_TYPE'].value_counts()

Committee                    320
Council                       96
Hearing/Committee             54
Council/Committee             39
Hearing                       19
Council/Hearing/Committee      3
Name: MEETING_TYPE, dtype: int64

In [6]:
# add holidays to 311 data
from datetime import date

import holidays

df['Holidays Created'] = np.nan

for x in holidays.CA(prov='AB',years=[2013,2014,2015,2016,2017]):
    df.loc[df['Date Created'] == x, 'Holidays Created'] = True
    
df['Holidays Created'].value_counts()

True    4016
Name: Holidays Created, dtype: int64

In [7]:
# merge weather and 311
#df2 = df2.add_suffix(' Created')
#df4 = pd.merge(df,df2,how='left',left_on='Date Created',right_on='date Created')
#df5 = pd.merge(df4,df2,how='left',left_on='Date Closed',right_on='date',suffixes=('_created','_closed'))

In [8]:
# merge with council data
df3 = df8.add_suffix(' Created')
df7 = pd.merge(df,df3,how='left',left_on='Date Created',right_on='MEETING_DATE Created')
#df7 = pd.merge(df6,df3,how='left',left_on='Date Closed',right_on='MEETING_DATE',suffixes=('_created','_closed'))

In [9]:
df7.columns

Index(['Reference Number', 'Date Created', 'Date Closed', 'Request Status',
       'Status Detail', 'Service Category', 'Service Code', 'Business Unit',
       'Neighbourhood', 'Community League', 'Ward', 'Address', 'Lat', 'Long',
       'Location', 'Ticket Source', 'Calendar Year', 'Days to Resolution',
       'Month Created', 'Month Closed', 'Day of Week Request Created',
       'Day of Week Request Closed', 'Holidays Created',
       'MEETING_DATE Created', 'MEETING_TYPE Created'],
      dtype='object')

In [10]:
df7['Business Unit'].value_counts(dropna=False)

Roadway Operations                             156642
Drainage Operations                             29281
Traffic Engineering                             26969
Forestry                                        23155
Parks                                           20383
Community Standards                             19207
Animal Control                                  10492
River Valley Operations                          2914
Pest                                             1687
Transit                                           494
Waste                                             180
Rangers/River Valley Trails                        99
Capitial City Clean Up - Community Services        13
Name: Business Unit, dtype: int64

In [11]:
df7.loc[df7['Business Unit'] == "Transit", 'Service Category']

729           Vandalism/Graffiti
960           Vandalism/Graffiti
1025      Traffic Lights & Signs
1147          Vandalism/Graffiti
1219          Vandalism/Graffiti
1333      Traffic Lights & Signs
3669      Traffic Lights & Signs
4229          Vandalism/Graffiti
4529          Vandalism/Graffiti
4992      Traffic Lights & Signs
5504          Vandalism/Graffiti
5688          Vandalism/Graffiti
7810      Traffic Lights & Signs
8678          Vandalism/Graffiti
8685          Vandalism/Graffiti
10371         Vandalism/Graffiti
13996         Vandalism/Graffiti
14100     Traffic Lights & Signs
15674     Traffic Lights & Signs
20351         Vandalism/Graffiti
20491         Vandalism/Graffiti
20843     Traffic Lights & Signs
21618     Traffic Lights & Signs
22887         Vandalism/Graffiti
22893         Vandalism/Graffiti
24055     Traffic Lights & Signs
24858         Vandalism/Graffiti
25540         Vandalism/Graffiti
25686         Vandalism/Graffiti
25908         Vandalism/Graffiti
          

In [12]:
df7['Service Category'].value_counts(dropna=False)

Pothole                            58577
Snow & Ice Maintenance             48635
Drainage Maintenance               32851
Traffic Lights & Signs             27155
Road/Sidewalk Maintenance          26018
Litter & Waste                     23784
Tree Maintenance                   23324
Parking Enforcement                18309
Parks & Sportsfield Maintenance    12854
Dead Animal Removal                10492
Vandalism/Graffiti                  6708
Pest Management                     1687
Structure Maintenance               1122
Name: Service Category, dtype: int64

In [13]:
del df7['Date Closed']
del df7['Request Status']
del df7['Ward']
del df7['Address']
del df7['Lat']
del df7['Long']
del df7['Location']
del df7['Calendar Year']
del df7['Month Closed']
del df7['Day of Week Request Closed']
#del df7['date Created']
del df7['MEETING_DATE Created']
del df7['Days to Resolution']

In [14]:
df7

,Reference Number,Date Created,Status Detail,Service Category,Service Code,Business Unit,Neighbourhood,Community League,Ticket Source,Month Created,Day of Week Request Created,Holidays Created,MEETING_TYPE Created
0,133345190,2013-01-02,NaN,Dead Animal Removal,Dead Animal,Animal Control,ROPER INDUSTRIAL,NaN,Telephone call,1,Wednesday,NaN,NaN
1,133350619,2013-01-02,NaN,Dead Animal Removal,Dead Animal,Animal Control,LARKSPUR,The Meadows Community League Association,Telephone call,1,Wednesday,NaN,NaN
2,133358214,2013-01-02,NaN,Dead Animal Removal,Dead Animal,Animal Control,GROVENOR,Grovenor Community League,Telephone call,1,Wednesday,NaN,NaN
3,133362915,2013-01-02,NaN,Dead Animal Removal,Dead Animal,Animal Control,KENSINGTON,Kensington Community League,Telephone call,1,Wednesday,NaN,NaN
4,133793810,2013-01-04,NaN,Dead Animal Removal,Dead Animal,Animal Control,RAMPART INDUSTRIAL,Cumberland/Oxford Community League (COCL),Telephone call,1,Friday,NaN,NaN
5,133801539,2013-01-04,NaN,Dead Animal Removal,Dead Animal,Animal Control,BONNIE DOON,The Cloverdale Community League,Telephone call,1,Friday,NaN,NaN
6,8007107432,2013-02-15,NaN,Pothole,Road,Roadway Operations,PARSONS INDUSTRIAL,NaN,Telephone call,2,Friday,NaN,NaN
7,8007110682,2013-02-16,NaN,Pothole,Road,Roadway Operations,SOUTHEAST INDUSTRIAL,NaN,Telephone call,2,Saturday,NaN,NaN
8,162275947,2014-10-10,NaN,Dead Animal Removal,Dead Animal,Animal Control,BELVEDERE,Belvedere Community League,Telephone call,10,Friday,NaN,NaN
9,8007119869,2013-02-18,NaN,Pothole,Road,Roadway Operations,CORONET INDUSTRIAL,NaN,Telephone call,2,Monday,True,NaN


In [15]:
df7 = df7.sort_values('Reference Number')
df7 = df7.replace('no', np.NaN)
df7 = df7.replace('below_freezing', np.NaN)

In [16]:
df7.columns

Index(['Reference Number', 'Date Created', 'Status Detail', 'Service Category',
       'Service Code', 'Business Unit', 'Neighbourhood', 'Community League',
       'Ticket Source', 'Month Created', 'Day of Week Request Created',
       'Holidays Created', 'MEETING_TYPE Created'],
      dtype='object')

In [17]:
del df7['Month Created']
del df7['Day of Week Request Created']

In [18]:
df7.to_csv('data/sequence_merged_created_no_weather.csv', index=False)

In [20]:
del df7['Community League']

In [21]:
df7

,Reference Number,Date Created,Status Detail,Service Category,Service Code,Business Unit,Neighbourhood,Ticket Source,Holidays Created,MEETING_TYPE Created
284799,133341338,2013-01-02,NaN,Vandalism/Graffiti,Shaw/Telus Box,Community Standards,WESTMOUNT,Internet Mail,NaN,NaN
0,133345190,2013-01-02,NaN,Dead Animal Removal,Dead Animal,Animal Control,ROPER INDUSTRIAL,Telephone call,NaN,NaN
1,133350619,2013-01-02,NaN,Dead Animal Removal,Dead Animal,Animal Control,LARKSPUR,Telephone call,NaN,NaN
2,133358214,2013-01-02,NaN,Dead Animal Removal,Dead Animal,Animal Control,GROVENOR,Telephone call,NaN,NaN
3,133362915,2013-01-02,NaN,Dead Animal Removal,Dead Animal,Animal Control,KENSINGTON,Telephone call,NaN,NaN
274894,133365471,2013-01-02,NaN,Dead Animal Removal,Dead Animal,Animal Control,HOLYROOD,Telephone call,NaN,NaN
284800,133562918,2013-01-03,NaN,Dead Animal Removal,Dead Animal,Animal Control,HIGHLANDS,Telephone call,NaN,NaN
284801,133720292,2013-01-03,NaN,Dead Animal Removal,Dead Animal,Animal Control,MCCAULEY,Telephone call,NaN,NaN
284802,133780574,2013-01-04,NaN,Vandalism/Graffiti,Newspaper Box/Sign,Community Standards,OLIVER,Telephone call,NaN,NaN
276418,133783648,2013-01-04,NaN,Dead Animal Removal,Dead Animal,Animal Control,DAVIES INDUSTRIAL EAST,Telephone call,NaN,NaN


In [22]:
df7.to_csv('data/sequence_merged_created_min.csv', index=False)